In [ ]:
import importlib

from ormatic.dao import *
from ripple_down_rules.orm_interface import *
from sqlalchemy import select
#python -m pytest -s test/test_relational_rdr.py::RelationalRDRTestCase::test_classify_scrdr
connection_string = "rdr@localhost:3306/RDR"  # os.getenv("RDR_DATABASE_URL")
engine = sqlalchemy.create_engine("mysql+pymysql://" + connection_string)
session = sqlalchemy.orm.Session(engine)
case_name = "" #dynamic import of case name
module_name = ".".join(case_name.split(".")[:-1])
case_type_name = case_name.split(".")[-1]
module = importlib.import_module(module_name)
# do this in the notebook
queried = session.scalar(select(get_dao_class(getattr(module, case_type_name))))
# the is the reconstructed case
case = queried.from_dao()

In [ ]:
#rendern svg
import inspect
import ipywidgets as widgets
from IPython.display import display
from IPython import get_ipython
from ripple_down_rules.user_interface.object_diagram import generate_object_graph

svg_output = widgets.HTML()

def render_svg(obj, name="Example"):
    graph = generate_object_graph(obj, name=name)
    svg_data = graph.pipe(format="svg").decode("utf-8")
    svg_output.value = svg_data

In [ ]:
#function
def my_rule_function(case, **kwargs):
    ...  # Your rule logic here

In [ ]:
import os
import time
import json
import IPython
import re

COMM_FILE = ""
TARGET_FUNC_NAME = "my_rule_function"

apply_rule_btn = widgets.Button(description="Apply Rule", button_style="success")
accept_rule_btn = widgets.Button(description="Accept Rule", button_style="primary")
toolbar = widgets.HBox([apply_rule_btn, accept_rule_btn])
toolbar.add_class('rdr-toolbar')

def on_apply_rule(btn):
    try:
        fn = globals().get(TARGET_FUNC_NAME)
        if callable(fn):
            fn(case)
            render_svg(case)
        else:
            print(f"Function '{TARGET_FUNC_NAME}' not found.")
    except Exception as e:
        print(f"Error applying function: {e}")

def send_cells_to_main_process(btn):
    print("Sending function back to main process...")
    try:
        ns = IPython.get_ipython().user_ns
        cells = ns.get('_ih') or ns.get('In') or []
        target = None
        pattern = re.compile(r'^\s*(async\s+)?def\s+' + re.escape(TARGET_FUNC_NAME) + r'\s*\(', flags=re.M)
        for content in reversed(cells):
            if isinstance(content, str) and pattern.search(content):
                target = content
                break
        if target:
            with open(COMM_FILE, 'w') as f:
                json.dump({'timestamp': time.time(), 'function_source': target}, f)
                f.flush()
                os.fsync(f.fileno())
            print(f"Function source written to: {COMM_FILE}")
        else:
            print("Function cell not found.")
    except Exception as e:
        print(f"Error writing function: {e}")

    # best-effort visualize
    try:
        on_apply_rule(btn)
    except Exception:
        pass

apply_rule_btn.on_click(on_apply_rule)
accept_rule_btn.on_click(send_cells_to_main_process)

display(toolbar)

In [ ]:
render_svg(case, name=case_name)
display(svg_output)

In [ ]:
from IPython.display import Image
Image(filename='./rule_tree.svg')